In [0]:
from pyspark.ml.regression import LinearRegression
from databricks.feature_store import FeatureStoreClient
from sparkdl.xgboost import XgboostRegressor
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
import mlflow
from pyspark.ml.feature import Imputer
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, translate
from pyspark.sql.functions import when
import warnings
import mlflow
import pandas as pd
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from sparkdl.xgboost import XgboostRegressor
from pyspark.sql.functions import log, col
from hyperopt import hp
from hyperopt import fmin, tpe, Trials
import numpy as np
from pyspark.ml.feature import Imputer
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, translate
from pyspark.sql.functions import when
import warnings
import mlflow
import pandas as pd
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from sparkdl.xgboost import XgboostRegressor
from pyspark.sql.functions import log, col
from hyperopt import hp
from hyperopt import fmin, tpe, Trials
import numpy as np
import mlflow.spark
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from databricks.feature_store import feature_table, FeatureLookup
from pyspark.sql.functions import monotonically_increasing_id, lit, expr, rand
import uuid
from databricks import feature_store
from pyspark.sql.types import StringType, DoubleType
from mlflow.tracking import MlflowClient
from databricks.feature_store import feature_table, FeatureLookup
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
import datetime
import warnings
warnings.filterwarnings(action='ignore')

In [0]:
def data_reading_table(table_name):
    raw_df=spark.sql(f'''select * from {table_name}''')
    return raw_df

In [0]:
def data_reading_table_select_col(table_name,select_col):
    raw_df=spark.sql(f'''select {select_col} from {table_name}'''.replace('"','').replace("'",''))
    return raw_df

In [0]:
def get_run_details(run):
    from mlflow.tracking import MlflowClient
    client = MlflowClient()
    experiment_id = run.info.experiment_id
    run_id = run.info.run_id
    runs_df = mlflow.search_runs(experiment_id, order_by=["attributes.start_time desc"], max_results=2)
    print(runs_df.display())
    return experiment_id,run_id

In [0]:
def get_best_model(run):
    from mlflow.tracking import MlflowClient
    client = MlflowClient()
    experiment_id = run.info.experiment_id
    run_id = run.info.run_id
    runs_df = mlflow.search_runs(experiment_id, order_by=["attributes.start_time desc"], max_results=2)
    print(runs_df.display())
    return experiment_id,run_id

In [0]:
def data_pipeline(df,x_col,y_col):
    categorical_cols = [field for (field, dataType) in df.dtypes if dataType == "string"]
    index_output_cols = [x + "Index" for x in categorical_cols]
    ohe_output_cols = [x + "OHE" for x in categorical_cols]
    
    string_indexer = StringIndexer(inputCols=categorical_cols, outputCols=index_output_cols, handleInvalid="skip")
    ohe_encoder = OneHotEncoder(inputCols=index_output_cols, outputCols=ohe_output_cols)
    numeric_cols = [field for (field, dataType) in df.dtypes if ((dataType == "double") & (field !=y_col))]
    assembler_inputs = ohe_output_cols + numeric_cols
    vec_assembler = VectorAssembler(inputCols=assembler_inputs, outputCol=x_col)
    
    return string_indexer,ohe_encoder,vec_assembler

In [0]:
def rf_model(params,x_col,y_col):
    max_depth=params[0]
    num_trees=params[1]
    max_bins=params[2]
    rf_params = {
            "maxDepth": max_depth,
            "numTrees": num_trees,
            "maxBins": max_bins
        }
        # Define model
    rf = RandomForestRegressor(featuresCol=x_col,labelCol=y_col, **rf_params)
    rf.setSeed(5678)
    return rf

In [0]:
def lr_model(regParam,x_col,y_col):
    lr_params = {
            "regParam": regParam
        }
        # Define model
    lr = LinearRegression(featuresCol=x_col,labelCol=y_col, **lr_params)
    return lr

In [0]:
def xgb_model(params,x_col,y_col):
    n_estimators=params[0]
    learning_rate=params[1]
    max_depth=params[2]
    xgb_params = {
         "n_estimators": n_estimators, 
        "learning_rate": learning_rate, 
        "max_depth": max_depth,
        "missing": 0,
        "nthread":4,
        "seed":5678
        }
        # Define model
    xgb = XgboostRegressor(featuresCol=x_col,labelCol=y_col,**xgb_params)
    return xgb

In [0]:
def data_split(data):
    train_df, test_df = (data.repartition(24).randomSplit([0.8, 0.2], seed=42))
    return train_df, test_df

In [0]:
def run_model(model_type,parma,train_df,test_df,model,x_col,y_col):
        string_indexer,ohe_encoder,vec_assembler=data_pipeline(train_df,x_col,y_col)
        #Pipeline
        pipeline = Pipeline(stages=[string_indexer, ohe_encoder, vec_assembler, model])
        # Fit the model
        pipeline_model=pipeline.fit(train_df)
        # make prediction 
        pred_df = pipeline_model.transform(test_df)
         # Evaluate and log metrics       
        regression_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol=y_col)
        r2 = regression_evaluator.setMetricName("r2").evaluate(pred_df)
        rmse = regression_evaluator.setMetricName("rmse").evaluate(pred_df)
        print(f"RMSE is {rmse}")
        print(f"R2 is {r2}")
        
        mlflow_log(model_type,parma,x_col,y_col,pipeline_model,rmse,r2)
        return train_df,test_df, model, pipeline,regression_evaluator

In [0]:
def load_model_and_predict(run_id,test_df):
    model_path = f"runs:/{run_id}/model"
    loaded_model = mlflow.spark.load_model(model_path)
    pred_df=loaded_model.transform(test_df)
    return pred_df

In [0]:
def mlflow_log(model_type,parma,x_col,y_col,pipeline_model,rmse,r2):
#         mlflow.log_param("label", x_col)
#         mlflow.log_param("features", y_col)
        mlflow.log_param("paramater",parma)
        mlflow.spark.log_model(pipeline_model, "model") 
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2", r2)
        if model_type=="Random Forest":
            mlflow.log_param("model type", model_type)
        elif model_type=="Linear Regression":
            mlflow.log_param("model type", model_type)
        elif model_type=="XGB":
            mlflow.log_param("model type", model_type)

In [0]:
def objective_function_rf(params):    

    # set the hyperparameters that we want to tune
    max_depth = params["max_depth"]
    num_trees = params["num_trees"]
    max_bins = params["max_bins"]

    with mlflow.start_run(experiment_id=experiment_id,run_name="tuning parmater for RF"):
        estimator = pipeline.copy({rf.maxDepth: max_depth, rf.numTrees: num_trees,rf.maxBins:max_bins})
        model = estimator.fit(train_df_t)
        preds = model.transform(test_df_t)
        rmse = regression_evaluator.evaluate(preds)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_param("paramater",params)
        mlflow.log_param("model type", "Random Forest")

    return rmse

In [0]:
def objective_function_lr(params):    
   
    # set the hyperparameters that we want to tune
    regParam = params["regParam"]

    with mlflow.start_run(experiment_id=experiment_id,run_name="tuning parmater for LR"):
        estimator = pipeline.copy({lr.regParam: regParam})
        model = estimator.fit(train_df_t)
        preds = model.transform(test_df_t)
        rmse = regression_evaluator.evaluate(preds)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_param("paramater",params)
        mlflow.log_param("model type", "Linear Regression")

    return rmse

In [0]:
def objective_function_xgb(params):    
    
    # set the hyperparameters that we want to tune
    n_estimators = int(params["n_estimators"])
    learning_rate = params["learning_rate"]
    max_depth = int(params["max_depth"])

    with mlflow.start_run(experiment_id=experiment_id,run_name="tuning parmater for XGB"):
        estimator = pipeline.copy({xgb.n_estimators: n_estimators, xgb.learning_rate: learning_rate,xgb.max_depth:max_depth})
        model = estimator.fit(train_df_t)
        preds = model.transform(test_df_t)
        rmse = regression_evaluator.evaluate(preds)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_param("paramater",params)
        mlflow.log_param("model type", "XGB")

    return rmse

In [0]:
def train(model_type,train_df,test_df,x_col,y_col,params,experiment_id):
    print('train baseline model>>>>>>>>>>>>>>')
    
    if model_type =="Linear Regression":
        model=lr_model(params,x_col,y_col)
        params_logs = {"regParam":params}
    elif model_type =="Random Forest":
        model=rf_model(params,x_col,y_col)
        params_logs = {"max_depth":params[0],"num_trees":params[1],"max_bins":params[2]}
    elif model_type =="XGB":
        model=xgb_model(params,x_col,y_col)
        params_logs = {"n_estimators":params[0],"learning_rate":params[1],"max_depth":params[2]}

    with mlflow.start_run(experiment_id=experiment_id,run_name=f'baseline model accuracy of {model_type}') as run:
        train_df, test_df, model, pipeline , regression_evaluator = run_model(model_type,params_logs,train_df,test_df,model,x_col,y_col)
    
    experiment_id,run_id = get_run_details(run)
    print("experiment_id :",experiment_id)
    print("run_id :",run_id)
    mlflow.end_run()

    ##predict with load model
    pred_df=load_model_and_predict(run_id,test_df)
    return train_df, test_df, model, pipeline , regression_evaluator,experiment_id,run_id

In [0]:
def tune(model_type,train_df,test_df,x_col,y_col,params):
    print ("tuning paramater in search space")
    if model_type =="Linear Regression":
        objective_function=objective_function_lr
    elif model_type =="Random Forest":
        objective_function=objective_function_rf
    elif model_type =="XGB":
        objective_function=objective_function_xgb

    print ("tuning paramater in search space")
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    num_evals = 5
    trials = Trials()
    best_hyperparam = fmin(fn=objective_function, 
                           space=search_space,
                           algo=tpe.suggest, 
                           max_evals=num_evals,
                           trials=trials,
                           rstate=np.random.default_rng(42))
    print ("tuning paramater end")
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    return best_hyperparam

In [0]:
# import mlflow

# # Set the tracking URI to Databricks
# mlflow.set_tracking_uri("databricks")

# # Set the name of the experiment
# experiment_name = "/Users/kexin@mz.co.kr/demo_mlflow"

# # Create the experiment
# mlflow.create_experiment(experiment_name)

In [0]:
experiment_id='4294012359381903'

In [0]:
def load_data(feature_table_name,dlt_table_name, feature_list ,lookup_key,y_col):
    raw_df=spark.sql(f'''select * from {dlt_table_name}''')
    ##inference random number column 추가: create_training_set 할때 inference_data_df 필수 파라미터고 왜 필수적인건지 알고 싶습니다.
    ##현재 inference은 feature store에 데이터 읽은 후에 추가하는 방식으로 되어 있는데 혹시 inference 컬럼은 gold table 만들때 추가 해도 되는지 알고 싶습니다.
    inference_data_df=raw_df.select(lookup_key, y_col,(rand() * 0.5-0.25).alias("rand_num"))
    
    fs=FeatureStoreClient()
    model_feature_lookups=[FeatureLookup(table_name=feature_table_name, feature_names =feature_list ,lookup_key=lookup_key)]
    df=fs.create_training_set(inference_data_df, model_feature_lookups, label=y_col, exclude_columns=lookup_key).load_df()
    
    return df

In [0]:
def register_baseline_model(model_name,run_id):
    model_uri = f"runs:/{run_id}/model"
    model_details = mlflow.register_model(model_uri=model_uri, name=model_name)

In [0]:
# %sql
# create or replace table airbnb_db.feature_store_tb(feature_id string,
# created_timestamp timestamp
# ) 